# Chains in LangChain

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
import os
import pandas as pd

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
df=pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### LLMChain

In [6]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
prompt=ChatPromptTemplate.from_template("What is the best name to desribe a company that makes {product}?")

In [7]:
chain=LLMChain(llm=llm,prompt=prompt)

In [8]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Linens" or "Regal Bedding" would be good names to describe a company that makes Queen Size Sheet Sets.'

### Simple Sequential Chains

In [9]:
from langchain.chains import SimpleSequentialChain

In [10]:
llm=ChatOpenAI(model=llm_model, temperature=0.9)

In [11]:
first_prompt=ChatPromptTemplate.from_template("What is the best name to describe \
a company that makes {product}?"
)

#Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [12]:
second_prompt=ChatPromptTemplate.from_template("Write a short 20 words description of company: {company_name}")
chain_two= LLMChain(llm=llm, prompt=second_prompt)

In [13]:
overall_simple_chain=SimpleSequentialChain(chains=[chain_one,chain_two], verbose=True)

In [14]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens Inc.
Royal Linens Inc. is a leading provider of luxury linens, bedding, and home décor items that elevate the comfort and style of any living space.

> Finished chain.


'Royal Linens Inc. is a leading provider of luxury linens, bedding, and home décor items that elevate the comfort and style of any living space.'

### Sequential Chains

In [1]:
from langchain.chains import SequentialChain

In [15]:
llm=ChatOpenAI(model=llm_model,temperature=0.9)

In [19]:
# prompt template 1: translate to english
prompt1=ChatPromptTemplate.from_template("Translate the following review to English:{Review}")

In [20]:
# chain 1: input= Review and output= English_Review
chain_1=LLMChain(llm=llm,prompt=prompt1,output_key="English_Review")

In [25]:
prompt2=ChatPromptTemplate.from_template(
        "Can you summarize the following review in 1 sentence: \
        {English_Review}"
    )

# chain 2: input= English_Review and output= summary
chain_2=LLMChain(llm=llm,prompt=prompt2,output_key="Summary")

In [27]:
# prompt template 3: find the language
prompt3=ChatPromptTemplate.from_template("What is the language of following review: {Review}")

# chain 3: input= Review and output= language
chain_3= LLMChain(llm=llm,prompt=prompt3,output_key="Language")

In [30]:
# prompt template 4: follow up message
prompt4=ChatPromptTemplate.from_template("Write a follow up response to the following summary in the specified language:\
                                        Summary: {Summary} \n\n \
                                        Language: {Language}")

chain_4= LLMChain(llm=llm,prompt=prompt4,output_key="followup_msg")

In [32]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
complete_chain=SequentialChain(chains= [chain_1,chain_2,chain_3,chain_4],
                               input_variables= ["Review"],
                               output_variables= ["English_Review","Summary","Language","followup_msg"])

In [33]:
review = df.Review[5]

In [34]:
complete_chain(review)

{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same in stores and the taste is much better... Old batch or counterfeit!?"',
 'Summary': "The reviewer finds the taste mediocre and suspects the possibility of an old batch or counterfeit product because the foam doesn't hold well.",
 'Language': 'The language of the review is French.',
 'followup_msg': "Réponse: Merci d'avoir partagé votre avis sur notre produit. Nous sommes désolés d'apprendre que vous avez trouvé le goût médiocre et que la mousse ne tient pas bien. Nous voulons vous assurer que notre produit est authentique et que nous prenons très au sérieux la qualité de nos produits. Avez-vous vérifié la date d'expiration ou la date de production sur le produit? Nous aimerions également en savoir plus sur votr

### Router Chains

In [35]:
physics_template = """You are a very smart physics professor. You are great at answering questions about physics in a concise\
and easy to understand manner. When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {  "name":"maths",
        "description": "Good for answering questions about maths", 
        "prompt_template": math_template
    },
    {  "name":"history",
        "description": "Good for answering questions about history", 
        "prompt_template": history_template
    },
    {
        "name": "computers", 
        "description": "Good for answering questions about physics", 
        "prompt_template": computerscience_template
    }
]

In [65]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [39]:
llm=ChatOpenAI(model=llm_model,temperature=0.9)

In [40]:
#destination_chains

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt=ChatPromptTemplate.from_template(prompt_template)
    chain=LLMChain(llm=llm,prompt=prompt)
    destination_chains[name]=chain
    


In [47]:

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations

['physics: Good for answering questions about physics',
 'maths: Good for answering questions about maths',
 'history: Good for answering questions about history',
 'computers: Good for answering questions about physics']

In [52]:
destinations_str = "\n".join(destinations)
destinations_str

'physics: Good for answering questions about physics\nmaths: Good for answering questions about maths\nhistory: Good for answering questions about history\ncomputers: Good for answering questions about physics'

In [45]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain=LLMChain(llm=llm, prompt=default_prompt)

In [63]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a description of what the prompt is best suited for. \
You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

< FORMATTING >: Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```
REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<Candidate Prompts>: {destinations}

<INPUT> : {{input}}

< OUTPUT (remember to include the ```json)>"""

In [67]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n< FORMATTING >: Return a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<Candidate Prompts>: physics: Good for answering questions about physics\nmaths: Good for answering questions ab

In [69]:
router_prompt=PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain=LLMRouterChain.from_llm(llm,router_prompt)

In [70]:
router_chain

LLMRouterChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], output_parser=RouterOutputParser(), template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n< FORMATTING >: Return a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications

In [72]:
chain= MultiPromptChain(router_chain=router_chain, 
                        destination_chains=destination_chains,
                        default_chain=default_chain,
                        verbose=True
                       )

In [73]:
chain.run("Who is Newton?")



> Entering new MultiPromptChain chain...
history: {'input': 'Who is Newton?'}
> Finished chain.


'Sir Isaac Newton was an English mathematician, physicist, and astronomer who lived from 1643 to 1727. He is famous for his laws of motion and universal gravitation, as well as his development of calculus. His work greatly influenced the scientific revolution and he is considered one of the most important scientists in history.'

In [74]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the type of electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that hits it and emits radiation at the maximum rate for a given temperature. The spectrum of black body radiation is continuous and depends solely on the temperature of the black body, with hotter black bodies emitting radiation at higher frequencies and with greater intensity. This type of radiation plays an important role in many areas of physics, including the study of stars, the development of quantum mechanics, and the understanding of the cosmic microwave background radiation.'

In [75]:
chain.run("How is chatgpt evolving via user input?")



> Entering new MultiPromptChain chain...
computers: {'input': 'How is the development of chatbots influenced by user input?'}
> Finished chain.


"As an AI language model, I can provide an answer to your question.\n\nThe development of chatbots is greatly influenced by user input. In fact, user input is the foundation on which chatbots are built. Chatbots are designed to interact with humans in a natural language format and therefore they must be able to understand and interpret user inputs accurately.\n\nThrough user input, developers can learn about user preferences, behaviors, and patterns. This information is used to tailor the chatbot's responses and improve its functionality, making it more useful and efficient.\n\nMoreover, user input also helps to identify areas where the chatbot needs improvement. This feedback is essential in the development process as it allows developers to identify potential bottlenecks or errors in the chatbot's design and make the necessary adjustments.\n\nTherefore, the development of chatbots is highly dependent on user input. It is important for developers to take user feedback seriously and us

In [76]:
chain.run("What are router chains in Langchain package in python?")



> Entering new MultiPromptChain chain...
computers: {'input': 'What are router chains in Langchain package in python?'}
> Finished chain.


'As an AI language model, I do not have personal experience with Langchain package in python, but I can provide an answer based on available information. \n\nThe Langchain package in python provides a way to handle the calling of multiple routers in a specific order, this is known as a router chain. Router chains are used in web development to handle routing in web-based projects. By using a router chain, developers can create a series of routes that have specific requirements before they can be accessed. This means that only certain users or certain conditions can access certain pages or parts of a website. The Langchain package in Python provides a flexible and intuitive way to handle router chains in web-based applications.'